In [1]:
import os 
import pandas as pd
import requests
import logging

from datetime import timedelta, datetime

In [2]:
day_log: str = datetime.now().strftime("%d_%m_%Y")
log_file_path = f"./logs/{day_log}.log"

log_file_path
logging.basicConfig(
    filename=log_file_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger()

In [2]:
input_folder: str = './data_source'
filename: str = 'Ventas_Consulta2024.csv'

base_dir = os.getcwd()  # Directorio actual en Jupyter o terminal
file_path = os.path.abspath(os.path.join(base_dir, input_folder, filename)) 
print(f"Intentando cargar el archivo desde: {file_path}")

Intentando cargar el archivo desde: c:\Users\LENOVO\Documents\repositories\cda_urrao\data_source\Ventas_Consulta2024.csv


In [13]:
COLUMNS: list = [
    "Fecha", 
    "Modal. Pago Contado", 
    "Modal. Pago Credito", 
    "Beneficiario Nombre", 
    "Doc. Fuente", 
    "Beneficiario Celular"
]

data = pd.read_csv(
    file_path,
    sep=";",
    encoding="utf-8",
    skip_blank_lines=True,
    engine="python"
).dropna(
    how="all"
)[COLUMNS]

In [14]:
data

,Fecha,Modal. Pago Contado,Modal. Pago Credito,Beneficiario Nombre,Doc. Fuente,Beneficiario Celular
0,2024/04/12,$0.00,"$427,300.00",G Y J FERRETERIAS S.A.,RTMYEC PL WOS455,3138864665
1,2024/05/18,"$283,755.00",$0.00,CALLE BURBANO JAVIER,RTMYEC PL MCK101,3152603151
2,2024/05/20,$0.00,"$428,000.00",ALCALDIA MUNICIPAL DE CHACHAGUI,RTMYEC PL OKN790,3113337993
3,2024/05/24,"$284,155.00",$0.00,VELASQUEZ GARCES DIEGO FERNANDO,RTMYEC PL HBZ702,3197049958
4,2024/05/27,$0.00,"$427,500.00",HARINERA DEL VALLE SA,RTMYEC PL SWO638,3127182640
...,...,...,...,...,...,...
2392,2024/12/31,"$284,155.00",$0.00,PORTILLA MARTINEZ ORLANDO HERNAN,RTMYEC UBV287,3122044954
2393,2024/12/31,"$283,755.00",$0.00,PORTILLA ARCOS ALVARO FERNANDO,RTMYEC FVM677,3155810345
2394,2024/12/31,"$284,155.00",$0.00,ESTRADA SANCHEZ JENNY SUSANA,RTMYEC KHX193,3007780138
2395,2024/12/31,"$193,397.00",$0.00,MARIN MURCIA LAURA CAMILA,RTMYEC FBH77G,3214267985


# Processor


In [15]:
import pandas as pd
import re
from datetime import datetime, timedelta


def extract_plate(doc_fuente: str) -> str:
    """
    Extrae la placa de un texto dado. Si no cumple el formato estándar,
    retorna el texto completo.
    
    Args:
        doc_fuente (str): El texto de la columna 'Doc. Fuente'.
    
    Returns:
        str: La placa extraída o el texto completo si no se detecta una placa válida.
    """
        
    pattern = r'\b[A-Z]{3}\d{3}\b|\b[A-Z]{3}\d{2}[A-Z]\b'
    match = re.findall(pattern, doc_fuente)
    if match:
        return match[0]
    return doc_fuente


def get_clients_for_messages(
        df: pd.DataFrame, 
        days_before: int = 3
    ) -> pd.DataFrame:
    
    df = df.copy()    
    current_date = pd.to_datetime(datetime.now().date())

    df["Fecha"] = pd.to_datetime(df["Fecha"], format="%Y/%m/%d").dt.date  # Convertir a date
    df["fecha_vencimiento"] = (pd.to_datetime(df["Fecha"]) + timedelta(days=365)).dt.date  # Convertir a date
    df["fecha_notificacion"] = (pd.to_datetime(df["Fecha"]) + timedelta(days=(365 - days_before))).dt.date  # Convertir a date
    df["dias_vencidos"] = (pd.to_datetime(df["fecha_notificacion"]) - pd.to_datetime(current_date)).dt.days
    
    df['Placa'] = df['Doc. Fuente'].apply(extract_plate)
    df["valid_number"] = df["Beneficiario Celular"].apply(lambda x: bool(re.match(r'^\d{10}$', str(x))))

    # Retornar solo los registros que cumplen la condición (cambiar a == 0 en producción)
    return df[df["dias_vencidos"] == 11].reset_index(drop=True)


def extract_message_parameters(df: pd.DataFrame) -> pd.DataFrame:
    required_columns = ["Beneficiario Celular", "Placa", "fecha_vencimiento", "valid_number"]
    
    if not all(col in df.columns for col in required_columns):
        raise ValueError("El DataFrame no tiene las columnas requeridas.")
    
    # Retornar solo las filas que tienen placa válida y fecha de vencimiento válida
    return df[["Beneficiario Nombre", "Beneficiario Celular", "Placa", "fecha_vencimiento", "valid_number"]]


# def extract_message_parameters(clients: pd.DataFrame) -> list:
#     parameters = []
#     for idx, row in clients.iterrows():
#         try:
#             nombre = row["Beneficiario Nombre"]
#             celular = row["Beneficiario Celular"]
#             placa = row["Placa"]
#             fecha_vencimiento = row["fecha_vencimiento"].strftime("%Y-%m-%d")

#             if pd.notna(celular) and pd.notna(placa) and pd.notna(fecha_vencimiento):
#                 parameters.append({
#                     "nombre": nombre,
#                     "celular": str(int(celular)),
#                     "placa": placa,
#                     "fecha_vencimiento": fecha_vencimiento
#                 })
        
#         except Exception as e:
#             print(f"Error procesando fila {idx}: {e}")
#             continue

#     return parameters


In [16]:
data_to_send = get_clients_for_messages(df=data)
data_to_send

,Fecha,Modal. Pago Contado,Modal. Pago Credito,Beneficiario Nombre,Doc. Fuente,Beneficiario Celular,fecha_vencimiento,fecha_notificacion,dias_vencidos,Placa,valid_number
0,2024-04-12,$0.00,"$427,300.00",G Y J FERRETERIAS S.A.,RTMYEC PL WOS455,3138864665,2025-04-12,2025-04-09,11,WOS455,True


In [7]:
data = [
    {
        "Beneficiario Nombre": "Juan Pérez",
        "Beneficiario Celular": "573175576781",
        "Doc. Fuente": "AWY21E",
        "fecha_vencimiento": datetime.now() + timedelta(days=13)
    },
    {
        "Beneficiario Nombre": "María García",
        "Beneficiario Celular": "573127088015",
        "Doc. Fuente": "XYZ123",
        "fecha_vencimiento": datetime.now() + timedelta(days=13)
    },
    {
        "Beneficiario Nombre": "Carlos Rodríguez",
        "Beneficiario Celular": "573205670292",
        "Doc. Fuente": "MNO456",
        "fecha_vencimiento": datetime.now() + timedelta(days=13)
    }
]

clients = pd.DataFrame(data)
clients

,Beneficiario Nombre,Beneficiario Celular,Doc. Fuente,fecha_vencimiento
0,Juan Pérez,573175576781,AWY21E,2025-04-11 14:59:12.129374
1,María García,573127088015,XYZ123,2025-04-11 14:59:12.129374
2,Carlos Rodríguez,573205670292,MNO456,2025-04-11 14:59:12.129374


In [17]:
import re

def extract_plate(doc_fuente: str) -> str:
    """
    Extrae la placa de un texto dado. Si no cumple el formato estándar,
    retorna el texto completo.
    
    Args:
        doc_fuente (str): El texto de la columna 'Doc. Fuente'.
    
    Returns:
        str: La placa extraída o el texto completo si no se detecta una placa válida.
    """
    # Patrón para identificar placas estándar (6 caracteres alfanuméricos)
    pattern = r'\b[A-Z]{3}\d{3}\b|\b[A-Z]{3}\d{2}[A-Z]\b'
    
    match = re.findall(pattern, doc_fuente)
    
    if match:
        # Si hay múltiples coincidencias, retorna la primera válida
        return match[0]
    else:
        # Si no se encuentra una placa válida, retornar el texto completo
        return doc_fuente

def process_plates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Procesa el DataFrame para extraer las placas de la columna 'Doc. Fuente'.
    
    Args:
        df (pd.DataFrame): DataFrame original.
    
    Returns:
        pd.DataFrame: DataFrame con la columna 'Placa' agregada.
    """
    df = df.copy()
    df['Placa'] = df['Doc. Fuente'].apply(extract_plate)
    return df


def extract_message_parameters(clients: pd.DataFrame) -> list:
    """
    Extrae los parámetros necesarios para enviar mensajes de WhatsApp desde un DataFrame.
    
    Args:
        clients (pd.DataFrame): DataFrame que contiene los datos procesados.
    
    Returns:
        list: Lista de diccionarios con los parámetros necesarios para cada mensaje.
    """
    parameters = []

    for idx, row in clients.iterrows():
        try:
            nombre = row["Beneficiario Nombre"]
            celular = row["Beneficiario Celular"]
            placa = row["Placa"]
            fecha_vencimiento = row["fecha_vencimiento"].strftime("%Y-%m-%d")  # Convertir a string con formato adecuado
            
            if pd.notna(celular) and pd.notna(placa) and pd.notna(fecha_vencimiento):
                parameters.append({
                    "nombre": nombre,
                    "celular": str(int(celular)),  # Convertir a string si viene como número
                    "placa": placa,
                    "fecha_vencimiento": fecha_vencimiento
                })
        
        except Exception as e:
            print(f"Error procesando fila {idx}: {e}")
            continue  # Continuar con el siguiente registro si hay algún problema

    return parameters


In [18]:
df_ok = process_plates(data_to_send)
df_ok

,Fecha,Modal. Pago Contado,Modal. Pago Credito,Beneficiario Nombre,Doc. Fuente,Beneficiario Celular,fecha_vencimiento,fecha_notificacion,dias_vencidos,Placa,valid_number
0,2024-04-12,$0.00,"$427,300.00",G Y J FERRETERIAS S.A.,RTMYEC PL WOS455,3138864665,2025-04-12,2025-04-09,11,WOS455,True


In [11]:
token_su = "EAANdZCNwsm0kBO0SHuGOyQXtZCST3kWl06Up5KueRST4aLYf6ZAV8wwR2eUmvVZAjSlEg70DNCnAQzeSGpbusnKEZBAbK6RnrXeDifPZAc7DcXZBsOIgS8kZAjYGZBH5H32oQKOvCQJGYxM6tg18jZA9lEFIE5CC6c0kCnzHLDmJvSgIzBgcPbgxuNynkZBZBmd5lmLvsgZDZD"

In [41]:
token = 'EAANdZCNwsm0kBOZCXDHFopLKxRontLTPABDM4ySy3puh2TZA4gYq5ZB3LC6EqVP6c9ZBq5Ai4OVo0ABUO3XZAwezGYCZAmJKhdguBk8baRPbZAbBiusbuAJW1RppZBMC1LZCVkRaZCFHtHOhGB6fs35ZClgOoOdNbLWxmYZAWoPoxTVzcHizW5NDy6ZBPJcHiOZBHV8guyOphbXNJoRgnvuBmUYeQkGeLMZD'
recipient_number = "573114481208"
phone_number_id = "628270670364121"
placa="ABC123"
fecha_vencimiento="2000-01-01"


In [ ]:
url = f"https://graph.facebook.com/v16.0/{phone_number_id}/messages"

headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

payload = {
    "messaging_product": "whatsapp",
    "to": recipient_number,
    "type": "template",
    "template": {
        "name": "",
        "language": {"code": "es_CO"},
        "components": [
            {
                "type": "body"
            }
        ]
    }
}

# Hacer la solicitud a la API
response = requests.post(url, headers=headers, json=payload)

# Mostrar la respuesta
print(response.status_code)
print(response.json())


200
{'messaging_product': 'whatsapp', 'contacts': [{'input': '573114481208', 'wa_id': '573114481208'}], 'messages': [{'id': 'wamid.HBgMNTczMTE0NDgxMjA4FQIAERgSMkU2MTRCQzM0NUFBQTQ0NDE2AA==', 'message_status': 'accepted'}]}
